In [ ]:
import numpy as np
import pandas as pd
from keras import layers, models
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, Adamax, SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from google.colab import files

# Esimerkki
'''

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

'''

# Ladataan data CSV-tiedostosta
file_path = "data_from_mysql_where_g16.csv"
data = pd.read_csv(file_path)

# Tarkastetaan datan rakenne
print(data.head())

# Suodatetaan pois kaikki rivit, joissa 'sensorvalue_d' on 0
data_filtered = data[data['sensorvalue_d'] != 0]

x_data = data_filtered[['sensorvalue_a', 'sensorvalue_b', 'sensorvalue_c']].values  # x, y, z
y_data = data_filtered['sensorvalue_d'].values # suunta

# Skaalataan syötteet [0, 1] väliin käyttäen StandardScaler
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data.astype('float32'))

# Suuntaa on 6 luokkaa
num_classes = 6
y_data = keras.utils.to_categorical(y_data - 1, num_classes)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

# Määritellään malli
model = keras.Sequential(
    [
        keras.Input(shape=(x_train.shape[1],)),  # Syötemuoto (x, y, z)
        layers.Dense(256, activation="relu"),  # Ensimmäinen tiheä kerros
        layers.Dropout(0.2),  # Dropout, jotta ylikoulutus ei tapahdu
        layers.Dense(128, activation="relu"),  # Toinen tiheä kerros
        layers.LeakyReLU(alpha=0.2),  # Vaihtoehto ReLU:lle
        layers.Dense(64, activation="relu"),  # Kolmas tiheä kerros
        layers.Dense(32, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),  # Lopullinen luokittelukerros
    ]
)

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Koulutetaan mallia
batch_size = 128
epochs = 100

optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[early_stopping])

# Arvioidaan malli testidatalla
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

   id            timestamp  groupid  from_mac  to_mac  sensorvalue_a  \
0  39  2024-11-14 12:30:54       16         0       0         1510.0   
1  40  2024-11-14 12:30:54       16         0       0         1503.0   
2  41  2024-11-14 12:30:54       16         0       0         1511.0   
3  42  2024-11-14 12:30:55       16         0       0         1509.0   
4  43  2024-11-14 12:30:56       16         0       0         1505.0   

   sensorvalue_b  sensorvalue_c  sensorvalue_d  sensorvalue_e  sensorvalue_f  
0         1405.0         1812.0            0.0            0.0              0  
1         1413.0         1818.0            0.0            0.0              0  
2         1416.0         1814.0            0.0            0.0              0  
3         1412.0         1814.0            0.0            0.0              0  
4         1410.0         1804.0            0.0            0.0              0  
x_train shape: (2210, 3)
x_test shape: (553, 3)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_55 (Dense)                     │ (None, 256)                 │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_56 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_11 (LeakyReLU)           │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_57 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_58 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_59 (Dense)                     │ (None, 6)                   │             198 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,454 (173.65 KB)

 Trainable params: 44,454 (173.65 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.3877 - loss: 1.6541 - val_accuracy: 0.5747 - val_loss: 1.2274
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5644 - loss: 1.1701 - val_accuracy: 0.6154 - val_loss: 0.9110
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6316 - loss: 0.8891 - val_accuracy: 0.6471 - val_loss: 0.8093
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6371 - loss: 0.8375 - val_accuracy: 0.6109 - val_loss: 0.8303
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6545 - loss: 0.7937 - val_accuracy: 0.6561 - val_loss: 0.7673
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6691 - loss: 0.7905 - val_accuracy: 0.6697 - val_loss: 0.7634
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6884 - loss: 0.7694 - val_accuracy: 0.6742 - val_loss: 0.7438
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6983 - loss: 0.7213 - val_accuracy: 0.6787 -